In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.decomposition import TruncatedSVD

In [3]:
df = pd.read_csv('dataset.csv')

E:\Application\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (5,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [79]:
pd_df = df[['user_device_composite_id', 'product_name', 'sessions','dt']];pd_df.head()

,user_device_composite_id,product_name,sessions,dt
0,NaN,NaN,NaN,2016-03-05
1,NaN,NaN,NaN,2016-03-05
2,NaN,NaN,NaN,2016-03-05
3,NaN,NaN,NaN,2016-03-05
4,NaN,NaN,NaN,2016-03-05


In [82]:
cleaned = pd_df[pd_df.product_name.notnull()];cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2344504 entries, 676880 to 4911959
Data columns (total 4 columns):
user_device_composite_id    object
product_name                object
sessions                    float64
dt                          object
dtypes: float64(1), object(3)
memory usage: 89.4+ MB


In [83]:
cleaned.head()

,user_device_composite_id,product_name,sessions,dt
676880,cad349ca827bd6d8f258ff77318cce02,3DSMAX2017,1.0,2016-07-16
676881,3d4193ded8caee7331cb60a96647e675,MAYA2017,1.0,2016-07-16
676889,cad349ca827bd6d8f258ff77318cce02,3DSMAX2017,3.0,2016-07-16
676890,cad349ca827bd6d8f258ff77318cce02,ACD2017,1.0,2016-07-16
676895,7c331fb3e409a97931e8883f169a44fb,RVT2017,3.0,2016-07-16


In [84]:
cleaned.sessions.describe()

count    2.344504e+06
mean     4.939621e+00
std      8.064950e+00
min      1.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      6.000000e+00
max      1.726000e+03
Name: sessions, dtype: float64

In [85]:
hotuser = cleaned[cleaned.sessions > 3];hotuser.sessions.describe()

count    1.027802e+06
mean     9.098987e+00
std      1.080619e+01
min      4.000000e+00
25%      5.000000e+00
50%      7.000000e+00
75%      1.000000e+01
max      1.726000e+03
Name: sessions, dtype: float64

In [86]:
hotusers = hotuser.drop(['sessions'],axis =1)

In [87]:
hotusers.head()

,user_device_composite_id,product_name,dt
676912,7c331fb3e409a97931e8883f169a44fb,3DSMAX2017,2016-07-16
676928,c6ffc131d931ee47674a1640c6e76c87,RECAP2017,2016-07-16
676958,dca6eed17f61e290801832a7c09b20a2,INVPROSA2017,2016-07-16
699160,0a44863e470c334a6359469060ac9d5a,ACD2017,2016-07-16
712019,dca6eed17f61e290801832a7c09b20a2,INVPROSA2017,2016-07-23


In [108]:
grouped = hotusers.groupby(['user_device_composite_id','dt']).agg({'product_name':lambda x:', '.join(x)}); grouped.head(20)

product_name
user_device_composite_id         dt                               
000009de373431343f4581dc27a6a60d 2017-05-27           INVPROSA2018
                                 2017-06-10  ACD2018, INVPROSA2018
                                 2017-06-24           INVPROSA2018
                                 2017-07-01           INVPROSA2018
                                 2017-07-22           INVPROSA2018
                                 2017-07-29           INVPROSA2018
                                 2017-09-16                ACD2018
                                 2017-10-14           INVPROSA2018
00000a166a2e45161981864dbe5a4128 2017-09-02                ACD2017
00002fc8fb7752d77f77b6a7810b361d 2017-07-29                RVT2017
000082ec4f7ee0f7ca12da0444dd0027 2017-09-16              CIV3D2018
00009862d9851523593fe0677a24ba10 2016-11-26                ACD2017
                                 2016-12-10                ACD2017
                                 2017-01-14                RVT2017
                                 2017-01-28                RVT2017
                                 2017-02-04                RVT2017
0000bd5068afc8465bd522a43f017cbe 2017-02-04                ACD2017
                                 2017-02-11                ACD2017
                                 2017-02-18                ACD2017
                                 2017-02-25                ACD2017

In [115]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 865519 entries, (000009de373431343f4581dc27a6a60d, 2017-05-27) to (fffe87109f270f2ecaff6bedc026dc26, 2017-10-21)
Data columns (total 1 columns):
product_name    865519 non-null object
dtypes: object(1)
memory usage: 12.0+ MB


In [116]:
grouped.unstack('dt')

product_name                        \
dt                                 2016-07-16 2016-07-23 2016-07-30   
user_device_composite_id                                              
000009de373431343f4581dc27a6a60d         None       None       None   
00000a166a2e45161981864dbe5a4128         None       None       None   
00002fc8fb7752d77f77b6a7810b361d         None       None       None   
000082ec4f7ee0f7ca12da0444dd0027         None       None       None   
00009862d9851523593fe0677a24ba10         None       None       None   
0000bd5068afc8465bd522a43f017cbe         None       None       None   
0001035cc2cb7ae1ae7f8e3a124d266a         None       None       None   
0002013ebebab3219b986ced83f45e13         None       None       None   
000233b3f53bdf0535765eb4869a3bc5         None       None       None   
0002404a15a28111970a728536de0f9b         None       None       None   
00028d260ed369ffee4f4ef120d950a9         None       None       None   
000357dc4298800f5b7a00ff1cd9229b         None       None       None   
00039f479a085a0492c167fd8696e8e3         None       None       None   
0003d4747c99beb679eef887e0902ca2         None       None       None   
000411574b98d2c65387198248d59a55         None       None       None   
00044379e4f88f2f49aaabbeacf91896         None       None       None   
0004a0e55abd8bf3090ad7d7fed47bb4         None       None       None   
0004bac69d2ea66d6347e3acfa185ee0         None       None       None   
000627f926ee8859173a21fa56b4ccf8         None       None       None   
00066008a5524595575a22f18fe6593e         None       None       None   
000680e446ea42a4cb5393b94bded655         None       None       None   
0006d0a7e1a1e9080ba175aed0d55772         None       None       None   
000772650867de8e0af2ba925b8b977f         None       None       None   
00077de3ba31809d64a2e5c2483e0443         None       None       None   
00084fbfcb786221449008c9a892bb71         None       None       None   
000872b8480da6ccadd69507c9b2af91         None       None       None   
000975ee7ecdfe490dea0f71fe22247d         None       None       None   
0009c08a504aa631b289615617f48daa         None       None       None   
000a82f9385b0c22838f5b2776620958         None       None       None   
000a8cf7d571c5c3ac257b9c8c332d0d         None       None       None   
...                                       ...        ...        ...   
fff2a7bed0bb8b0b18da19e99240a67e         None       None       None   
fff2bed5b951bdf88b1b3bf6af161eea         None       None       None   
fff3047dcc1f7de1bd5fa89503d71c14         None       None       None   
fff3412eadbf12c1cee1aefc29480b10         None       None       None   
fff358659065e846a7a8fe5202c05777         None       None       None   
fff3c9eb255a2feb0b32d10882b953f5         None       None       None   
fff3d2e0fa25b2e44ea8434a10650a7b         None       None       None   
fff4d9bf44988298d1c172f0cf910d81         None       None       None   
fff54261866f3e47da57f241034388d2         None       None       None   
fff5b4001b1930b738fc189f8bdb7a84         None       None       None   
fff6314453f142663f84cb088c798de3         None       None       None   
fff6a779e97b85120f308c0c8013b512         None       None       None   
fff6efeb351e502f4fe53b0ec620945d         None       None       None   
fff6fd03ef429311655983f70cd44bf0         None       None       None   
fff7bc73a898738257a2044783f20b2c         None       None       None   
fff7c00f139025806dd0e8171e564dd5         None       None       None   
fff801c3a85dd8d95979f5dd3aefdfb7         None       None       None   
fff9ae3adf069ec7c48a44160220b52f         None       None       None   
fff9d63bdb64bc7945bfd7310d7d0a7a         None       None       None   
fffa12d9488d44bc0b46f544b6cfcfbc         None       None       None   
fffa2659d5813f4188f079542e23adf2         None       None       None   
fffa3d8df25b3d1c85fb21468447f978         None       None       None   
fffa5b28def9616da4cd10e84b8b53fc         None       None 

In [117]:
lifecycle = grouped.unstack('dt')

In [118]:
lifecycle.index.is_unique

True

In [119]:
lifecycle.to_csv('lifecycle.csv',sep=',')

In [89]:
#idxed = hotusers.set_index(['user_device_composite_id', 'dt'], append=True );idxed.head()

,,,product_name
,user_device_composite_id,dt,
676912,7c331fb3e409a97931e8883f169a44fb,2016-07-16,3DSMAX2017
676928,c6ffc131d931ee47674a1640c6e76c87,2016-07-16,RECAP2017
676958,dca6eed17f61e290801832a7c09b20a2,2016-07-16,INVPROSA2017
699160,0a44863e470c334a6359469060ac9d5a,2016-07-16,ACD2017
712019,dca6eed17f61e290801832a7c09b20a2,2016-07-23,INVPROSA2017


In [93]:
#idxed.index.is_unique

True